**Mount Google Drive locally**

Mount your Google Drive on your runtime using an authorization code.

Note this only supports reading, writing, and moving files; to programmatically modify sharing settings or other metadata, use one of the other options below.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import cv2
import numpy as np
import random
import os
from imutils import paths
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers

# initialize the data and labels
data = []
labels = []

dataset_path = "/content/drive/My Drive/Deep Learning with Python/Day 1/dataset"
IMAGE_DIMS = (224, 224, 3)

In [0]:
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(dataset_path)))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for ind, imagePath in enumerate(imagePaths):
	# load the image, pre-process it, and store it in the data list
	print("%d\t%s\tLABEL %s"%(ind, imagePath, imagePath.split(os.path.sep)[-2]))
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)

	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

In [0]:
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [0]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(train_images, test_images, train_labels, test_labels) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

In [0]:
# Create the network
network = models.Sequential()
network.add(layers.Conv2D(32, (3, 3), activation="relu", padding='same', input_shape=train_images.shape[1:]))
network.add(layers.Flatten())
network.add(layers.Dense(100, activation='relu'))
network.add(layers.Dense(5, activation='softmax'))

network.compile(optimizer='adam',
				loss='categorical_crossentropy',
                metrics=['accuracy'])


In [0]:
history = network.fit(train_images, train_labels, epochs=50, batch_size=32, verbose=2)

In [0]:
test_loss, test_acc = network.evaluate(test_images, test_labels, verbose=2)
print('test_acc:', test_acc)